## 1. Init


In [ ]:
from google.colab import drive
import os
import shutil
import subprocess

drive.mount('/content/drive')

CODE_FOLDER = 'code'
DATA_FOLDER = 'data'
TEST_FOLDER = 'test'
PROJECT_DIR = '/content/drive/MyDrive/Text2SVG'
CODE_PATH = f"{PROJECT_DIR}/{CODE_FOLDER}"
DATA_PATH = f"{PROJECT_DIR}/{DATA_FOLDER}"
TEST_PATH = f"{PROJECT_DIR}/{TEST_FOLDER}"


## 2. Clone Code Repository


In [ ]:
# Clean existing code directory and clone fresh repository
if os.path.exists(CODE_PATH):
    shutil.rmtree(CODE_PATH)

os.chdir(PROJECT_DIR)
result = subprocess.run(['git', 'clone', 'https://github.com/huanbasara/dual-branch-vae.git', CODE_FOLDER], 
                      capture_output=True, text=True)

if result.returncode != 0:
    raise Exception(f"Failed to clone repository: {result.stderr}")

print(f"✅ Code repository cloned successfully to {CODE_PATH}")


## 3. Install Dependencies

In [ ]:
%pip install torch torchvision numpy scipy pandas scikit-learn matplotlib pillow svglib svgpathtools
%pip install kornia opencv-python cairosvg pyyaml easydict tqdm
# Install additional packages for SVG rendering support
%pip install svglib reportlab cssutils

In [ ]:
# Add code path to Python sys.path so we can import our modules
import sys
if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)
    print(f"✅ Added {CODE_PATH} to Python path")
else:
    print(f"✅ {CODE_PATH} already in Python path")


In [ ]:
# Simple test: try to import pydiffvg_lite
try:
    import pydiffvg_lite
    print("✅ pydiffvg_lite import successful!")
except Exception as e:
    print(f"❌ pydiffvg_lite import failed: {e}")


In [ ]:
# Test specific SVG file: aardvark
svg_file = f"{DATA_PATH}/aardvark/66543-200.svg"
print(f"Using test SVG: {svg_file}")

if os.path.exists(svg_file):
    try:
        # Test svg_to_scene function
        canvas_width, canvas_height, shapes, shape_groups = pydiffvg_lite.svg_to_scene(svg_file)
        
        print(f"✅ SVG parsed successfully:")
        print(f"   Canvas: {canvas_width} x {canvas_height}")
        print(f"   Shapes: {len(shapes)}")
        print(f"   Groups: {len(shape_groups)}")
        
        # Get first shape's coordinate points
        if len(shapes) > 0:
            first_shape = shapes[0]
            print(f"   Shape type: {type(first_shape).__name__}")
            if hasattr(first_shape, 'points'):
                points = first_shape.points
                print(f"   Points: {points.shape}")
                print(f"   First points: {points[:5]}")
            
        print("✅ SVG to coordinates conversion successful!")
        
    except Exception as e:
        print(f"❌ SVG processing failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"❌ SVG file not found: {svg_file}")


In [ ]:
# Test reading SVG files from data folder and convert to coordinate points
import os

# Look for SVG files in data folder structure
print(f"Looking for SVG files in: {DATA_PATH}")

# Find first SVG file
svg_file = None
for root, dirs, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith('.svg'):
            svg_file = os.path.join(root, file)
            print(f"Found SVG file: {svg_file}")
            break
    if svg_file:
        break

if svg_file:
    try:
        # Test svg_to_scene function
        canvas_width, canvas_height, shapes, shape_groups = pydiffvg_lite.svg_to_scene(svg_file)
        
        print(f"✅ SVG parsed successfully:")
        print(f"   Canvas size: {canvas_width} x {canvas_height}")
        print(f"   Number of shapes: {len(shapes)}")
        print(f"   Number of shape groups: {len(shape_groups)}")
        
        # Get first shape's coordinate points
        if len(shapes) > 0:
            first_shape = shapes[0]
            print(f"   First shape type: {type(first_shape).__name__}")
            if hasattr(first_shape, 'points'):
                points = first_shape.points
                print(f"   Points shape: {points.shape}")
                print(f"   First few points: {points[:5]}")
            
        print("✅ SVG to coordinates conversion successful!")
        
    except Exception as e:
        print(f"❌ SVG processing failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No SVG files found in data folder")
